# Прогнозирование цен потребительского ритейла по тестовой выборке на основе глубоких нейронных сетей
__Выполнил:__ *Домченко Максим*

__Студент группы:__ *РИМ-130962*

In [ ]:
# Подключение Google Drive к Colab
from pathlib import Path
from google.colab import drive

drive.mount('/content/drive')
ROOT = Path('/content/drive/MyDrive')

# Настройка структуры папок проекта
PROJECT_DIR = ROOT / 'price_forecasting'
DATA = PROJECT_DIR / 'data'
RAW = DATA / 'raw'
PROCESSED = DATA / 'processed'
MODELS = PROJECT_DIR / 'models_test_1_epoch'
MODELS.mkdir(exist_ok=True)

print('Проектная папка подключена:', PROJECT_DIR)

Mounted at /content/drive
Проектная папка подключена: /content/drive/MyDrive/price_forecasting


In [ ]:
!pip install pytorch-forecasting

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.7/197.7 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.0/819.0 kB 59.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 127.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 99.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 66.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 110.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# Основные библиотеки
import numpy as np
import pandas as pd

# Визуализация
import matplotlib.pyplot as plt
import seaborn as sns

# Метрики
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error, r2_score

# Pytorch и TFT
import torch
from pytorch_forecasting import TimeSeriesDataSet, GroupNormalizer
from torch.utils.data import DataLoader  # Исправлено здесь
from pytorch_forecasting.models import TemporalFusionTransformer

# Прочее
import pickle

# Версии библиотек и проверка GPU
print("Версия Pandas:", pd.__version__)
print("Версия NumPy:", np.__version__)
print("Версия PyTorch:", torch.__version__)
print("Доступна GPU:", torch.cuda.is_available())

Версия Pandas: 2.2.2
Версия NumPy: 2.0.2
Версия PyTorch: 2.6.0+cu124
Доступна GPU: True


In [ ]:
# Загрузка данных
train_df = pd.read_parquet(PROCESSED / 'train_features_final.parquet')
val_df = pd.read_parquet(PROCESSED / 'val_features_final.parquet')
test_df = pd.read_parquet(PROCESSED / 'test_features_final.parquet')

# Выбор магазина (например, CA_1)
store_id = 'CA_1'
train_df = train_df[train_df['store_id'] == store_id]
val_df = val_df[val_df['store_id'] == store_id]
test_df = test_df[test_df['store_id'] == store_id]

# Проверка размеров
print(f"Размер train_df: {train_df.shape}")
print(f"Размер val_df: {val_df.shape}")
print(f"Размер test_df: {test_df.shape}")

Размер train_df: (3673873, 22)
Размер val_df: (599263, 22)
Размер test_df: (600503, 22)


In [ ]:
# Выведем признаки и их типы данных
print("Список признаков и их типы данных:")
for col in train_df.columns:
    print(f"{col}: {train_df[col].dtype}")

Список признаков и их типы данных:
store_id: category
item_id: category
wm_yr_wk: int64
sell_price: float64
date: datetime64[ns]
month: int32
year: int64
event_name_1: object
event_type_1: object
event_name_2: object
event_type_2: object
snap_CA: int64
snap_TX: int64
snap_WI: int64
day_of_week: int32
is_weekend: int64
log_sell_price: float64
event_flag: int64
event_type_National: int64
event_type_Cultural: int64
event_type_Religious: int64
event_type_Sporting: int64


In [ ]:
# Приводим категориальные признаки к строковому типу
for df in [train_df, val_df, test_df]:
    df['month'] = df['month'].astype(str)
    df['year'] = df['year'].astype(str)
    df['event_name_1'] = df['event_name_1'].astype(str)
    df['event_type_1'] = df['event_type_1'].astype(str)
    df['snap_CA'] = df['snap_CA'].astype(str)
    df['day_of_week'] = df['day_of_week'].astype(str)
    df['is_weekend'] = df['is_weekend'].astype(str)

# Определение таргета и признаков
target = 'log_sell_price'

# Признаки для модели
time_idx = 'date'
group_ids = ['store_id', 'item_id']  # добавлен store_id для корректности группировки
categorical_features = [
    'month', 'year', 'day_of_week', 'is_weekend',
    'event_name_1', 'event_type_1', 'snap_CA'
]

# Числовые признаки без sell_price
numerical_features = [
    'event_flag', 'event_type_National', 'event_type_Cultural',
    'event_type_Religious', 'event_type_Sporting'
]

# Финальный список признаков
features = group_ids + categorical_features + numerical_features

print("Таргет:", target)
print("Категориальные признаки:", categorical_features)
print("Числовые признаки:", numerical_features)
print("Все признаки:", features)

Таргет: log_sell_price
Категориальные признаки: ['month', 'year', 'day_of_week', 'is_weekend', 'event_name_1', 'event_type_1', 'snap_CA']
Числовые признаки: ['event_flag', 'event_type_National', 'event_type_Cultural', 'event_type_Religious', 'event_type_Sporting']
Все признаки: ['store_id', 'item_id', 'month', 'year', 'day_of_week', 'is_weekend', 'event_name_1', 'event_type_1', 'snap_CA', 'event_flag', 'event_type_National', 'event_type_Cultural', 'event_type_Religious', 'event_type_Sporting']


In [ ]:
# Убедимся, что дата отсортирована по 'date'
train_df = train_df.sort_values(by='date')
val_df = val_df.sort_values(by='date')
test_df = test_df.sort_values(by='date')

# Подготовим индексы времени для модели
time_idx = 'time_idx'
for df in [train_df, val_df, test_df]:
    df[time_idx] = (df['date'] - train_df['date'].min()).dt.days

# Проверим промежуток времени в данных
print("Диапазон дат в train:", train_df['date'].min(), train_df['date'].max())
print("Диапазон дат в val:", val_df['date'].min(), val_df['date'].max())
print("Диапазон дат в test:", test_df['date'].min(), test_df['date'].max())

# Проверка итоговых данных
train_df.head()

In [ ]:
!pip install pytorch-forecasting pytorch-lightning

In [ ]:
# Определение разумных длин окон
max_encoder_length = 60
max_prediction_length = 28

print(f"Максимальная длина окна кодировщика: {max_encoder_length}")
print(f"Максимальная длина окна прогнозирования: {max_prediction_length}")

# Удаление пропусков (если они есть)
train_df.dropna(inplace=True)
val_df.dropna(inplace=True)
test_df.dropna(inplace=True)

# Проверим число записей по каждой группе
group_counts = train_df.groupby(group_ids, observed=True).size()
valid_groups = group_counts[group_counts >= (max_encoder_length + max_prediction_length)].index

train_df_filtered = train_df.set_index(group_ids).loc[valid_groups].reset_index()
val_df_filtered = val_df.set_index(group_ids).loc[valid_groups].reset_index()
test_df_filtered = test_df.set_index(group_ids).loc[valid_groups].reset_index()

print(f"Размер train_df до фильтрации: {train_df.shape}")
print(f"Размер train_df после фильтрации: {train_df_filtered.shape}")
print(f"Число групп после фильтрации: {train_df_filtered.groupby(group_ids, observed=True).ngroups}")


Максимальная длина окна кодировщика: 60
Максимальная длина окна прогнозирования: 28
Размер train_df до фильтрации: (3673873, 23)
Размер train_df после фильтрации: (3672914, 23)
Число групп после фильтрации: 3020


In [ ]:
from pytorch_forecasting.data.encoders import NaNLabelEncoder

# Создание датасета для обучения
training = TimeSeriesDataSet(
    train_df_filtered,
    time_idx=time_idx,
    target=target,
    group_ids=group_ids,
    max_encoder_length=max_encoder_length,
    min_encoder_length=max_encoder_length // 2,
    max_prediction_length=max_prediction_length,
    min_prediction_length=1,
    static_categoricals=group_ids,
    time_varying_known_categoricals=categorical_features,
    time_varying_known_reals=['time_idx'],
    time_varying_unknown_reals=[target] + numerical_features,
    target_normalizer=GroupNormalizer(groups=group_ids),
    categorical_encoders={col: NaNLabelEncoder(add_nan=True) for col in categorical_features + group_ids},
    add_relative_time_idx=True,
    add_target_scales=True,
    add_encoder_length=True,
    allow_missing_timesteps=True
)

In [ ]:
# Создание датасетов для валидации и теста
validation = TimeSeriesDataSet.from_dataset(
    training, val_df_filtered, predict=True, stop_randomization=True
)
testing = TimeSeriesDataSet.from_dataset(
    training, test_df_filtered, predict=True, stop_randomization=True
)

# Создание DataLoader'ов
batch_size = 12288
num_workers = 12

train_dataloader = training.to_dataloader(train=True, batch_size=batch_size, num_workers=num_workers)
val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size, num_workers=num_workers)
test_dataloader = testing.to_dataloader(train=False, batch_size=batch_size, num_workers=num_workers)

# Проверка загрузки данных
x, y = next(iter(train_dataloader))
print("Размерность батча (x):", x['encoder_cont'].shape)
print("Размерность таргета (y):", y[0].shape)

In [ ]:
from pytorch_forecasting.metrics import MAE, RMSE, MAPE

tft = TemporalFusionTransformer.from_dataset(
    training,
    learning_rate=0.03,
    hidden_size=64,
    attention_head_size=2,
    dropout=0.2,
    hidden_continuous_size=32,
    loss=RMSE(),
    log_interval=10,
    optimizer="Adam",  # заменил Ranger на Adam
    reduce_on_plateau_patience=4,
)

print(f"Количество параметров модели: {(tft.size()/1e3):.1f}K")

/usr/local/lib/python3.11/dist-packages/lightning/pytorch/utilities/parsing.py:209: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
/usr/local/lib/python3.11/dist-packages/lightning/pytorch/utilities/parsing.py:209: Attribute 'logging_metrics' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['logging_metrics'])`.


Количество параметров модели: 502.2K


In [ ]:
from lightning.pytorch import Trainer
from lightning.pytorch.callbacks import EarlyStopping, LearningRateMonitor

early_stop_callback = EarlyStopping(monitor="val_loss", patience=10, verbose=True, mode="min")
lr_logger = LearningRateMonitor()

trainer = Trainer(
    max_epochs=1,
    accelerator='gpu',
    devices=[0],
    precision='32-true',
    enable_model_summary=True,
    gradient_clip_val=0.1,
    callbacks=[early_stop_callback, lr_logger]
)

torch.backends.cudnn.enabled = False

trainer.fit(
    tft,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader
)

INFO: Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
INFO:lightning.pytorch.utilities.rank_zero:Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: You are using a CUDA device ('NVIDIA A100-SXM4-40GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO: Metric val_loss improved. New best score: 0.259
INFO:lightning.pytorch.callbacks.early_stopping:Metric val_loss improved. New best score: 0.259
INFO: `Trainer.fit` stopped: `max_epochs=1` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=1` reached.


In [ ]:
# Сохранение модели в файл
model_path = MODELS / 'tft_model_limited_dataset.ckpt'
trainer.save_checkpoint(model_path)

print(f"Модель успешно сохранена в {model_path}")

Модель успешно сохранена в /content/drive/MyDrive/price_forecasting/models_test_1_epoch/tft_model_limited_dataset.ckpt


In [ ]:
# ─────────────────────────────────────────────────────────────
# Загрузка сохранённой TFT-модели
# ─────────────────────────────────────────────────────────────
from pytorch_forecasting.models import TemporalFusionTransformer

model_path = MODELS / "tft_model_limited_dataset.ckpt"

# map_location='cpu' гарантирует, что весы попадут в ОЗУ,
# даже если GPU недоступна/не нужна
tft_loaded = TemporalFusionTransformer.load_from_checkpoint(
    model_path,
    map_location="cpu",
)
tft_loaded.eval()

print(f"Модель успешно загружена из: {model_path}")

In [ ]:
# ─────────────────────────────────────────────────────────────
# Сквозная функция для метрик
# ─────────────────────────────────────────────────────────────
from sklearn.metrics import (
    mean_absolute_error,
    mean_squared_error,
    mean_absolute_percentage_error,
    r2_score,
)
import numpy as np

def calc_metrics(y_true: np.ndarray, y_pred: np.ndarray) -> dict:
    """
    Возвращает словарь базовых регрессионных метрик.
    Все входы должны быть одномерными numpy-массивами одинаковой длины.
    """
    return {
        "MAE":  mean_absolute_error(y_true, y_pred),
        "MSE":  mean_squared_error(y_true, y_pred),
        "RMSE": np.sqrt(mean_squared_error(y_true, y_pred)),
        "MAPE": mean_absolute_percentage_error(y_true, y_pred),
        "R2":   r2_score(y_true, y_pred),
    }

In [15]:
# ════════════════════════════════════════════════════════════════
#  Расчёт метрик (train / val / test) на GPU с жёсткой очисткой
#  Переменные, которые уже должны существовать:
#      • tft_loaded  – TFT-модель, загруженная из .ckpt
#      • training / validation / testing – TimeSeriesDataSet
#      • MODELS      – pathlib.Path к каталогу для сохранения результатов
# ════════════════════════════════════════════════════════════════
import gc, json, pickle, torch
from pathlib import Path
from torch.utils.data import DataLoader
from pytorch_forecasting.data import TimeSeriesDataSet
from sklearn.metrics import (
    mean_absolute_error, mean_squared_error,
    mean_absolute_percentage_error, r2_score,
)

# ──────────────────────────────
# ПАРАМЕТРЫ
# ──────────────────────────────
DEVICE      = "cuda"     # "cpu" <- безопасно, "cuda" <- быстрее
BATCH_SIZE  = 40000      # если словили CUDA OOM — уменьшить

tft_loaded.to(DEVICE).eval()
torch.cuda.empty_cache()

# ──────────────────────────────
# ВСПОМОГАТЕЛЬНЫЕ ФУНКЦИИ
# ──────────────────────────────
def move_to_device(obj, device: str = DEVICE):
    "Рекурсивно переносит тензоры / коллекции тензоров на устройство."
    if torch.is_tensor(obj):
        return obj.to(device, non_blocking=True)
    if isinstance(obj, dict):
        return {k: move_to_device(v, device) for k, v in obj.items()}
    if isinstance(obj, (list, tuple)):
        return type(obj)(move_to_device(v, device) for v in obj)
    return obj  # числа, строки, None …

def extract_pred_tensor(out):
    "Извлекает сам тензор предсказаний из любого формата выхода модели."
    if torch.is_tensor(out):
        return out
    if isinstance(out, dict):
        return out.get("prediction", out.get("output"))
    if hasattr(out, "prediction"):
        return out.prediction
    if hasattr(out, "output"):
        return out.output
    if isinstance(out, (list, tuple)) and len(out):
        return out[0]
    raise TypeError(f"Неизвестный формат выхода: {type(out)}")

def to_loader(ds, batch_size=BATCH_SIZE):
    "Гарантированно получаем DataLoader с таргетом."
    if isinstance(ds, DataLoader):
        return ds
    if isinstance(ds, TimeSeriesDataSet):
        return ds.to_dataloader(
            train=False,
            batch_size=min(batch_size, len(ds)),
            shuffle=False,
            num_workers=0,
            pin_memory=True,
        )
    raise TypeError(f"Ожидался TimeSeriesDataSet/DataLoader, а не {type(ds)}")

@torch.no_grad()
def get_pred_and_true(model, dataset):
    loader = to_loader(dataset)
    pred_buf, true_buf = [], []

    for batch in loader:
        x, y = batch[0], batch[1]

        x = move_to_device(x)
        y = move_to_device(y)

        raw_out = model(x)
        preds   = extract_pred_tensor(raw_out)

        pred_buf.append(preds.squeeze(-1).detach().cpu())
        if isinstance(y, (list, tuple)):
            y = y[0]
        true_buf.append(y.squeeze(-1).detach().cpu())

        # оперативно чистим GPU-память
        del x, y, raw_out, preds
        torch.cuda.empty_cache()

    y_pred = torch.cat(pred_buf).numpy().ravel()
    y_true = torch.cat(true_buf).numpy().ravel()

    del pred_buf, true_buf
    gc.collect(); torch.cuda.empty_cache()

    return y_pred, y_true

# ──────────────────────────────
# РАССЧИТЫВАЕМ МЕТРИКИ
# ──────────────────────────────
datasets = {
    "train": training,
    "val"  : validation,
    "test" : testing,
}

metrics_dict = {}
for split, ds in datasets.items():
    print(f"Получаем прогноз для {split}…")
    y_pred, y_true = get_pred_and_true(tft_loaded, ds)
    metrics_dict[f"{split}_metrics"] = calc_metrics(y_true, y_pred)

# ──────────────────────────────
# ВЫВОД НА ЭКРАН
# ──────────────────────────────
print("\nИтоговые метрики:")
for split, m in metrics_dict.items():
    print(f"\n{split}:")
    for k, v in m.items():
        print(f"  {k}: {v:.4f}")

Получаем прогноз для train…
Получаем прогноз для val…
Получаем прогноз для test…

Итоговые метрики:

train_metrics:
  MAE: 0.2249
  MSE: 0.1254
  RMSE: 0.3541
  MAPE: 163809180254208.0000
  R2: 0.6292

val_metrics:
  MAE: 0.1976
  MSE: 0.0672
  RMSE: 0.2591
  MAPE: 0.1435
  R2: 0.7793

test_metrics:
  MAE: 0.1977
  MSE: 0.0675
  RMSE: 0.2599
  MAPE: 0.1414
  R2: 0.7773


In [16]:
# ─────────────────────────────────────────────────────────────
# Сериализация результатов
# ─────────────────────────────────────────────────────────────
import pickle

metrics_pkl = MODELS / "tft_model_limited_dataset_metrics.pkl"
with open(metrics_pkl, "wb") as f:
    pickle.dump(metrics_dict, f)

print(f"\nМетрики сохранены:\n  • {metrics_pkl}")


Метрики сохранены:
  • /content/drive/MyDrive/price_forecasting/models_test_1_epoch/tft_model_limited_dataset_metrics.pkl


In [17]:
# Проверка метрик
for dataset, metrics in metrics_dict.items():
    print(f"\nМетрики для {dataset}:")
    for key, value in metrics.items():
        print(f"{key}: {value:.4f}")


Метрики для train_metrics:
MAE: 0.2249
MSE: 0.1254
RMSE: 0.3541
MAPE: 163809180254208.0000
R2: 0.6292

Метрики для val_metrics:
MAE: 0.1976
MSE: 0.0672
RMSE: 0.2591
MAPE: 0.1435
R2: 0.7793

Метрики для test_metrics:
MAE: 0.1977
MSE: 0.0675
RMSE: 0.2599
MAPE: 0.1414
R2: 0.7773
